In [ ]:
from torch.utils.data import Dataset, DataLoader, random_split
import torch
import pandas as pd
from nn import model, activations, optimizers, losses
from nn.layers import Dense
from nn.activations import Sigmoid, ReLU
class CSVDataset(Dataset):
    def __init__(self, filepath):
        df = pd.read_csv(filepath)
        self.X = df[["x0", "x1"]].values.astype("float32")
        self.y = df["y"].values.astype("int")

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        x = torch.tensor(self.X[idx])
        y = torch.tensor(self.y[idx])
        return x, y


dataset = CSVDataset("nn_data.csv")

random_generator = torch.Generator().manual_seed(42)

train_data, test_data = random_split(dataset, [int(0.8*len(dataset)), int(0.2*len(dataset))], generator=random_generator)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True, generator=random_generator)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


In [ ]:
"""model = torch.nn.Sequential(
    torch.nn.Linear(2, 25),
    torch.nn.ReLU(),
    torch.nn.Linear(25, 10),
    torch.nn.ReLU(),
    torch.nn.Linear(10, 1),
    torch.nn.Sigmoid()
)"""

In [ ]:
model = model.Sequential([
    Dense(2, 25),
    ReLU(),
    Dense(25, 10),
    ReLU(),
    Dense(10, 1)
])

loss_fn = losses.BinaryCrossEntropyWithLogits()